
# BIOINFORMATIKA - deceptív fitneszfüggvény gyakorlati alkalmazása
* **Feladat 1:** 4 bites, teljesen deceptív DF2 függvény bevezetése
* **Feladat 2:** (opcionális): 8 bites implementáció

---

Deceptív függvény: Olyan fitness függvény, amelyben a lokálisan növekvő sémák (schema-average fitness) a GA-t a globális optimumtól távolodó irányba vezetik, így a GA jellemzően rossz lokális optimumhoz konvergál.



## PROJEKT FELADATTAL SZEMBENI ELVÁRÁSOK

1. Konstruáljunk 4 bites reprezentációra olyan fitneszfüggvényt, ahol a
globális maximum a 0000-ban van, de bármely szkémára teljesül, hogyha az összes 0-t kicseréljük 1-esre akkor a szkéma fitnesze nőni fog (ez egy deceptív függvény)
2.   Bizonyítsuk is példánk helyességét
3.   Röviden ismertessük algoritmusunk lépéseit

## PSEUDO KÓD: 4 bites deceptív függvény

Alábbiakban adok egy pseudó kódot, ami lefedi a 4 bites deceptív függvény implementációját (de ettől el lehet térni). Bejárja az összes alacsonyabb rendű sémát (van benne legalább egy 0 és egy *). Minden ilyen sémára kiszámolja: (1) az eredeti átlagfitneszt, (2) a „0→1-re cserélt” szkéma átlagfitneszét, (3) ellenőrzi, hogy mindig nő-e → ezzel igazolja a deceptív tulajdonságot.

MEGJEGYZÉS: A lentiek alapján össze lehet állítani a saját implementációt. Kérem a megoldást magyarázatokkal is ellátni, vagy törekedjünk arra, hogy beszédes legyen a kód.



```
---------------------------------------------
DEFINÍCIÓ: 4 bites DF2 fitness tábla (Whitley /extra info csatolt doksiban)
---------------------------------------------
DF2 = {
  "0000" → 28,  "0001" → 26,  "0010" → 24,  "0011" → 18,
  "0100" → 22,  "0101" →  6,  "0110" → 14,  "0111" →  0,
  "1000" → 20,  "1001" → 12,  "1010" → 10,  "1011" →  2,
  "1100" →  8,  "1101" →  4,  "1110" →  6,  "1111" → 30
}
```
```
---------------------------------------------
FÜGGVÉNY BIT_NOT(bits)
---------------------------------------------
  # Bitenkénti negálás: 0 ↔ 1
  eredmény ← üres string
  CIKLUS i = 1 .. HOSSZ(bits)
    HA bits[i] = '0' AKKOR
        eredmény ← eredmény + '1'
    KÜLÖNBEN
        eredmény ← eredmény + '0'
    VÉGE HA
  VÉGE CIKLUS
  VISSZA eredmény
VÉGE FÜGGVÉNY
```
```
---------------------------------------------
ÚJ DECEPTÍV FÜGGVÉNY DEFINÍCIÓJA
---------------------------------------------
# g(x) = DF2(¬x)
# Minden 4 bites b-re:
DECEPTIVE_F egy asszociatív tömb (szótár)

CIKLUS MINDEN b ∈ {0,1}^4 BITSTRING
    y ← BIT_NOT(b)
    DECEPTIVE_F[b] ← DF2[y]
VÉGE CIKLUS
```
```
---------------------------------------------
SEGÉDFÜGGVÉNY: összes 4 bites bitstring
---------------------------------------------
BITSTRINGS ← összes 4 hosszúságú 0/1 string felsorolása
  # pl. 0000, 0001, ..., 1111

```
```
---------------------------------------------
FÜGGVÉNY STRINGS_IN_SCHEMA(schema)
---------------------------------------------
  # Visszaadja a sémához illeszkedő bitstringeket
  eredmény_lista ← üres lista

  CIKLUS MINDEN s ∈ BITSTRINGS
    illeszkedik ← IGAZ
    CIKLUS i = 1 .. 4
      c ← schema[i]
      sc ← s[i]
      HA c ≠ '*' ÉS c ≠ sc AKKOR
          illeszkedik ← HAMIS
          MEGSZAKÍT CIKLUS
      VÉGE HA
    VÉGE CIKLUS

    HA illeszkedik = IGAZ AKKOR
        HOZZÁAD eredmény_lista-hoz s
    VÉGE HA
  VÉGE CIKLUS

  VISSZA eredmény_lista
VÉGE FÜGGVÉNY
```
```
---------------------------------------------
FÜGGVÉNY AVG_FITNESS_OF_SCHEMA(schema)
---------------------------------------------
  xs ← STRINGS_IN_SCHEMA(schema)
  összeg ← 0
  N ← elemszám(xs)

  CIKLUS MINDEN x ∈ xs
    összeg ← összeg + DECEPTIVE_F[x]
  VÉGE CIKLUS

  VISSZA összeg / N
VÉGE FÜGGVÉNY
```
```
---------------------------------------------
FÜGGVÉNY FLIP_ZEROS_TO_ONES(schema)
---------------------------------------------
  # Minden '0' → '1'; '*' és '1' marad
  eredmény ← üres string
  CIKLUS i = 1 .. 4
    HA schema[i] = '0' AKKOR
        eredmény ← eredmény + '1'
    KÜLÖNBEN
        eredmény ← eredmény + schema[i]
    VÉGE HA
  VÉGE CIKLUS
  VISSZA eredmény
VÉGE FÜGGVÉNY
```

```
---------------------------------------------
FŐPROGRAM (main)
---------------------------------------------
ELJÁRÁS MAIN()

  KIÍR "=== 4 bites deceptív fitneszfüggvény ==="
  KIÍR "Bitstring   Fitness"
  KIÍR "-------------------"

  # 1) Táblázat kiírása
  CIKLUS MINDEN b ∈ BITSTRINGS rendezett sorrendben
    KIÍR b, DECEPTIVE_F[b]
  VÉGE CIKLUS

  # 2) Globális maximum keresése
  best_val ← maximum(DECEPTIVE_F értékei közül)
  best_points ← { b | DECEPTIVE_F[b] = best_val }

  KIÍR "Globális maximum érték:", best_val
  KIÍR "Globális maximum helye(i):", best_points

  # 3) Szkéma-ellenőrzés
  KIÍR "=== Szkéma-ellenőrzés (alacsonyabb rendű sémák) ==="

  symbols ← ['0', '1', '*']
  violations ← üres lista

  # Végigmegyünk az összes {0,1,*}^4 sémán
  CIKLUS MINDEN schema ∈ symbols^4
    HA schema = "****" AKKOR
        FOLYTAT következő sémára
    VÉGE HA

    HA '0' NINCS BENNE schema-BAN AKKOR
        FOLYTAT (nincs mit lecserélni)
    VÉGE HA

    HA '*' NINCS BENNE schema-BAN AKKOR
        FOLYTAT (teljesen specifikus, pl. 0000 – ezt nem vizsgáljuk)
    VÉGE HA

    schema_flipped ← FLIP_ZEROS_TO_ONES(schema)

    avg_orig ← AVG_FITNESS_OF_SCHEMA(schema)
    avg_flip ← AVG_FITNESS_OF_SCHEMA(schema_flipped)

    HA NEM (avg_flip > avg_orig) AKKOR
        HOZZÁAD violations-hoz (schema, schema_flipped, avg_orig, avg_flip)
    VÉGE HA

  VÉGE CIKLUS

  HA violations ÜRES AKKOR
     KIÍR "Minden vizsgált sémára teljesül:"
     KIÍR "  a 0→1 csere növeli a szkéma átlagos fitneszét."
  KÜLÖNBEN
     KIÍR "Találtunk ellenpéldá(ka)t:"
     CIKLUS MINDEN (s, sf, a, b) ∈ violations
        KIÍR s, "->", sf, ": átlag", a, "->", b
     VÉGE CIKLUS
  VÉGE HA

VÉGE ELJÁRÁS
```


## SAJÁT IMPLEMENTÁCIÓ BEMUTATÁSA

```
# This is formatted as code
```



In [1]:
DF2 = {
  "0000": 28,  "0001": 26,  "0010": 24,  "0011": 18,
  "0100": 22,  "0101":  6,  "0110": 14,  "0111":  0,
  "1000": 20,  "1001": 12,  "1010": 10,  "1011":  2,
  "1100":  8,  "1101":  4,  "1110":  6,  "1111": 30
}

def BIT_NOT(bits):
  """Bitenkénti negálás: 0 ↔ 1"""
  result = ""
  for bit in bits:
    if bit == '0':
      result += '1'
    else:
      result += '0'
  return result

def generate_bitstrings(length):
    """Generálja az összes adott hosszúságú bitstringet."""
    if length == 0:
        return [""]

    smaller_bitstrings = generate_bitstrings(length - 1)

    result = []
    for bs in smaller_bitstrings:
        result.append("0" + bs)
        result.append("1" + bs)
    return sorted(result) # A konzisztens sorrend érdekében

# Generálja az összes 4 bites bitstringet
BITSTRINGS = generate_bitstrings(4)

# Új deceptív függvény definíciója: g(x) = DF2(¬x)
DECEPTIVE_F = {}
for b in BITSTRINGS:
    y = BIT_NOT(b)
    DECEPTIVE_F[b] = DF2[y]

def STRINGS_IN_SCHEMA(schema):
  """Visszaadja a sémához illeszkedő bitstringeket."""
  result_list = []
  for s in BITSTRINGS:
    matches = True
    for i in range(len(schema)): # Pythonban az indexelés 0-tól indul
      c = schema[i]
      sc = s[i]
      if c != '*' and c != sc:
        matches = False
        break
    if matches:
      result_list.append(s)
  return result_list

def AVG_FITNESS_OF_SCHEMA(schema):
  """Kiszámolja egy séma átlagos fitneszét."""
  xs = STRINGS_IN_SCHEMA(schema)
  total_sum = 0
  N = len(xs)

  if N == 0: # Megelőzzük a nullával való osztást
      return 0

  for x in xs:
    total_sum += DECEPTIVE_F[x]
  return total_sum / N

def FLIP_ZEROS_TO_ONES(schema):
  """Minden '0' → '1'; '*' és '1' marad."""
  result = ""
  for char in schema:
    if char == '0':
      result += '1'
    else:
      result += char
  return result

def MAIN():
  print("=== 4 bites deceptív fitneszfüggvény ===")
  print("Bitstring   Fitness")
  print("-------------------")

  # 1) Táblázat kiírása
  for b in BITSTRINGS: # BITSTRINGS már rendezett
    print(f"{b}           {DECEPTIVE_F[b]}")

  # 2) Globális maximum keresése
  best_val = -float('inf') # Negatív végtelen mint kezdeti minimum érték
  best_points = []

  if DECEPTIVE_F: # Ellenőrizzük, hogy nem üres-e
      best_val = max(DECEPTIVE_F.values())

  for b, fitness in DECEPTIVE_F.items():
      if fitness == best_val:
          best_points.append(b)

  print(f"\nGlobális maximum érték: {best_val}")
  print(f"Globális maximum helye(i): {', '.join(best_points)}")

  # 3) Szkéma-ellenőrzés
  print("\n=== Szkéma-ellenőrzés (alacsonyabb rendű sémák) ===")

  symbols = ['0', '1', '*']
  violations = []

  import itertools
  # Végigmegyünk az összes {0,1,*}^4 sémán
  all_schemas = [''.join(p) for p in itertools.product(symbols, repeat=4)]

  for schema in all_schemas:
    if schema == "****":
        continue

    if '0' not in schema: # Nincs mit lecserélni
        continue

    # Ha nincs '*' benne, akkor ez egy teljesen specifikus bitstring (pl. "0000"),
    # ezeket a pszeudokód szerint nem vizsgáljuk alacsonyabb rendű sémaként.
    if '*' not in schema:
        continue

    schema_flipped = FLIP_ZEROS_TO_ONES(schema)

    avg_orig = AVG_FITNESS_OF_SCHEMA(schema)
    avg_flip = AVG_FITNESS_OF_SCHEMA(schema_flipped)

    if not (avg_flip > avg_orig): # Ha nincs szigorú növekedés
        violations.append((schema, schema_flipped, avg_orig, avg_flip))

  if not violations: # Ha a violations lista üres
     print("Minden vizsgált sémára teljesül:")
     print("  a 0→1 csere növeli a szkéma átlagos fitneszét.")
  else:
     print("Találtunk ellenpéldá(ka)t:")
     for s, sf, a, b in violations:
        print(f"{s} -> {sf}: átlag {a} -> {b}")

# Főprogram futtatása
MAIN()


=== 4 bites deceptív fitneszfüggvény ===
Bitstring   Fitness
-------------------
0000           30
0001           6
0010           4
0011           8
0100           2
0101           10
0110           12
0111           20
1000           0
1001           14
1010           6
1011           22
1100           18
1101           24
1110           26
1111           28

Globális maximum érték: 30
Globális maximum helye(i): 0000

=== Szkéma-ellenőrzés (alacsonyabb rendű sémák) ===
Minden vizsgált sémára teljesül:
  a 0→1 csere növeli a szkéma átlagos fitneszét.


Rendben, itt van a Python implementáció a 4 bites deceptív fitneszfüggvényhez, a megadott pszeudokód alapján:

A 4 bites deceptív fitneszfüggvény implementációja sikeresen lefutott. Az eredmények a következők:

Fitness táblázat: Kiírta az összes 4 bites bitstringhez tartozó fitness értéket.
Globális maximum: A globális maximum fitness érték 30, ami a "0000" bitstringhez tartozik.
Szkéma-ellenőrzés: Az ellenőrzés megerősítette, hogy minden vizsgált alacsonyabb rendű sémára teljesül, hogy a '0' bitek '1'-re cserélése növeli a séma átlagos fitneszét. Ez igazolja a függvény deceptív tulajdonságát, azaz a GA-t a "0000" optimumtól távolodó irányba tereli. Azaz teljesült az első projektfeladat.

## MAGYARÁZAT:

*   ..
*   ..
*   ..
*   ..

## EXTRA PROJEKT FELADAT – 8 bites implementáció

*   Itt ugyanaz a logika, de két 4 bites deceptív blokk összegével.
*   Az alábbi pszeudokód a 4 bites verzióra épít, tehát feltételezi, hogy a korábbi Deceptive4Fitness és segédfüggvények már rendelkezésre állnak, így azokat kérem figyelembe venni az implementáció során


Emlékeztető (már megvan):
*  DF2 4 bites táblázat
*  BIT_NOT(bits)
*  DECEPTIVE_F[b] = DF2(BIT_NOT(b))
*  Deceptive4Fitness(x) = DECEPTIVE_F[x]


Most **erre építjük az 8 bites blokk-szintű függvényt**.

## 8 bites deceptív függvény – két 4 bites blokk összege

```
---------------------------------------------
FÜGGVÉNY Deceptive4Fitness(x4)
---------------------------------------------
  # x4: 4 bites string
  # Feltételezzük, hogy a 4 bites DECEPTIVE_F táblázat
  # már definiálva van a korábbiak szerint
  VISSZA DECEPTIVE_F[x4]
VÉGE FÜGGVÉNY
```

```
---------------------------------------------
FÜGGVÉNY Deceptive8Fitness(x8)
---------------------------------------------
  # x8: 8 bites bináris string

  blokk1 ← x8[0..3]    # első 4 bit
  blokk2 ← x8[4..7]    # második 4 bit

  f1 ← Deceptive4Fitness(blokk1)
  f2 ← Deceptive4Fitness(blokk2)

  VISSZA f1 + f2       # két blokk összegzett fitnesse
VÉGE FÜGGVÉNY
```

## Összes 8 bites bitstring és globális maximum keresése
```
---------------------------------------------
BITSTRINGS_8 ← összes 8 bites 0/1 string
  # pl. 00000000, 00000001, ..., 11111111
---------------------------------------------

ELJÁRÁS EllenőrizGlobálisMaximum8bit()
  best_val ← -∞
  best_points ← üres lista

  CIKLUS MINDEN x ∈ BITSTRINGS_8
      f ← Deceptive8Fitness(x)
      HA f > best_val AKKOR
          best_val ← f
          best_points ← [x]
      EGYÉBKÉNT HA f = best_val AKKOR
          HOZZÁAD x-ET best_points-hoz
      VÉGE HA
  VÉGE CIKLUS

  KIÍR "8 bites globális maximum érték:", best_val
  KIÍR "8 bites globális maximum helye(i):", best_points
ELJÁRÁS VÉGE
```
Elméletileg tudjuk, hogy:

* 4 biten a globális maximum 0000
* ezért 8 biten a globális maximum 00000000,
mert mindkét blokk külön-külön 0000-nál maximális, így az összeg is ott maximális.

## Szkémák 8 biten – segédfüggvények
Most ugyanezt a sémavizsgálatot kiterjesztjük 8 bitre:
* séma: 8 hosszú string {0,1,*} karakterekkel.

```
---------------------------------------------
FÜGGVÉNY StringsInSchema8(schema8)
---------------------------------------------
  # schema8: pl. "0***1**0", hossza 8
  eredmény_lista ← üres lista

  CIKLUS MINDEN s ∈ BITSTRINGS_8
      illeszkedik ← IGAZ
      CIKLUS i = 1 .. 8
          c  ← schema8[i]
          sc ← s[i]
          HA c ≠ '*' ÉS c ≠ sc AKKOR
              illeszkedik ← HAMIS
              MEGSZAKÍT belső CIKLUS
          VÉGE HA
      VÉGE CIKLUS

      HA illeszkedik = IGAZ AKKOR
          HOZZÁAD s-t eredmény_lista-hoz
      VÉGE HA
  VÉGE CIKLUS

  VISSZA eredmény_lista
VÉGE FÜGGVÉNY
```
```
---------------------------------------------
FÜGGVÉNY AvgFitnessOfSchema8(schema8)
---------------------------------------------
  xs ← StringsInSchema8(schema8)
  összeg ← 0
  N ← elemszám(xs)

  CIKLUS MINDEN x ∈ xs
      összeg ← összeg + Deceptive8Fitness(x)
  VÉGE CIKLUS

  VISSZA összeg / N
VÉGE FÜGGVÉNY
```
```
---------------------------------------------
FÜGGVÉNY FlipZerosToOnesSchema(schema)
---------------------------------------------
  # Általános 8 bites sémára: 0 → 1, 1 / * változatlan
  eredmény ← üres string
  CIKLUS i = 1 .. HOSSZ(schema)
      HA schema[i] = '0' AKKOR
          eredmény ← eredmény + '1'
      KÜLÖNBEN
          eredmény ← eredmény + schema[i]
      VÉGE HA
  VÉGE CIKLUS
  VISSZA eredmény
VÉGE FÜGGVÉNY
```

## Végül implementáljuk a szkémaszintű ellenőrzést 8 biten
Most végigmegyünk az összes {0,1,*}^8 sémán, és megmutatjuk, hogy amelyikben van 0 és van *, ott a 0→1 csere növeli az átlagfitneszt.

```
---------------------------------------------
ELJÁRÁS SchemaEllenőrzés8bit()
---------------------------------------------
  symbols ← ['0', '1', '*']
  violations ← üres lista

  # Végigmegyünk az összes 8 hosszú sémán
  CIKLUS MINDEN schema8 ∈ symbols^8

      HA schema8 = "********" AKKOR
          FOLYTASD a következő sémával
      VÉGE HA

      HA '0' NINCS BENNE schema8-BAN AKKOR
          FOLYTASD (nincs mit lecserélni)
      VÉGE HA

      HA '*' NINCS BENNE schema8-BAN AKKOR
          FOLYTASD (teljesen specifikus, pl. 00000000 – ezt nem vizsgáljuk)
      VÉGE HA

      schema_flipped ← FlipZerosToOnesSchema(schema8)

      avg_orig ← AvgFitnessOfSchema8(schema8)
      avg_flip ← AvgFitnessOfSchema8(schema_flipped)

      HA NEM (avg_flip > avg_orig) AKKOR
          # Ha nincs szigorú növekedés, akkor ellentmondás a deceptivitásnak
          HOZZÁAD (schema8, schema_flipped, avg_orig, avg_flip) elemet violations-hoz
      VÉGE HA

  VÉGE CIKLUS

  HA violations ÜRES AKKOR
      KIÍR "Minden vizsgált 8 bites sémára teljesül:"
      KIÍR "  a 0→1 csere növeli a szkéma átlagos fitneszét."
  EGYÉBKÉNT
      KIÍR "Találtunk ellenpéldá(ka)t a deceptív tulajdonságra:"
      CIKLUS MINDEN (s, sf, a, b) ∈ violations
          KIÍR s, " -> ", sf, ": átlag", a, "->", b
      VÉGE CIKLUS
  VÉGE HA
ELJÁRÁS VÉGE

```

## Főprogram 8 bitre így alakul
```
ELJÁRÁS MAIN_8()
  # 1) 4 bites blokk init (DF2 + bit negálás) – ahogy a 4 bites feladatban
  # 2) BITSTRINGS_8 generálása
  # 3) EllenőrizGlobálisMaximum8bit()
  # 4) SchemaEllenőrzés8bit()

  EllenőrizGlobálisMaximum8bit()
  SchemaEllenőrzés8bit()
ELJÁRÁS VÉGE
```


A 8 bites verzió lényege összefoglalva:
* építőkocka szemlélettel a 4 bites deceptív függvényre épül,
* formálisan megadja Deceptive8Fitness(x)-et,
* sémaszinten is bizonyítja, hogy a 0→1 csere az átlagfitneszt növeli ⇒
a GA-t „rossz irányba” húzza, miközben a globális optimum 00000000.

## SAJÁT 8 bites IMPLEMENTÁCIÓ BEMUTATÁSA

```
# This is formatted as code
```


## 8 BITES IMPLEMENTÁCIÓHOZ MAGYARÁZAT:

*   ..
*   ..
*   ..
*   ..